# Project data analisys m1

##### Problmeas encontrados y cómo se resolvieron
### Readme con fotos e instrucciones
Importar librerías
### Adquisition
Importar datos, sql y api
### Wrangling
Limpieza de datos. Tengo dos nulos: 192(191) y 98(99)
### Analisys
Crear funciones siempre mejor

Clacular distancias y limpiar

Asignar la station y address a places

### Reporting
Two options: all place or one

To csv

Additional documents: pdf, power points,...

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import re
import os
import duckdb
from shapely.geometry import Point
import geopandas as gpd
from modules import geo_calculations
pd.options.display.max_columns = None
pd.options.display.max_rows = None

| Place of interest | Type of place (*) | Place address | BiciMAD station | Station location |

---
### Gardens

Agrupar en funciones module adquisition

In [2]:
# endpoint api Madrid gardens
response = requests.get('https://datos.madrid.es/egob/catalogo/200761-0-parques-jardines.json')
results_api = response.json()

In [3]:
# Selecciono la lista que quiero
list_api_garden = results_api['@graph']

In [4]:
# Transformo a data frame
df_garden = pd.DataFrame(list_api_garden)

In [5]:
# Separo langitud y latitud de la columna location
df_garden_location = df_garden["location"].apply(pd.Series)

In [6]:
# Separa los elementos de address
df_garden_address = df_garden["address"].apply(pd.Series)

In [7]:
# Elimino lo innecesario para hacerlo más legible de @type
# La barra se podría cambiar con un str.replace por una coma
df_garden_type = df_garden["@type"].str.replace('https://datos.madrid.es/egob/kos/entidadesYorganismos/', '')

In [8]:
# En el data frame de garden, me quedo solo con title, para no repetir @type y evitar posibles problemas en el futuro
both_df_garden = pd.concat([df_garden[['title']], df_garden_location, df_garden_address, df_garden_type], axis = 1)

In [9]:
# Elimino columnas o selecciono columnas que quiero: location, type, address and title of gardens, o no
cleaning_df_garden = both_df_garden[['title', '@type', 'locality', 'postal-code', 'street-address', 'latitude', 'longitude']]

---
### Bicimad

Extraer funcion de adquisition

In [10]:
# endpoint duck

In [11]:
db_bicimad = duckdb.connect('bicimad1.db')

In [12]:
# Data Frame data base statiosn

In [13]:
df_bicimad = db_bicimad.sql('select * from bicimad_stations').df()

In [14]:
# Separo geometry coordinates

In [15]:
df_bicimad[['longitude', 'latitude']] = df_bicimad["geometry.coordinates"].apply(lambda x: pd.Series(str(x).replace('[','').replace(']','').split(",")))

In [16]:
df_bicimad = df_bicimad[['name', 'address', 'latitude', 'longitude',]]

---

## Muestro los dos resultados de data wrangling

In [17]:
#df_bicimad

In [18]:
#cleaning_df_garden

## Data Analisys

In [19]:
# Incluir de alguna forma la comparacion con un loop o comprehension

In [20]:
# Incluyo las funciones para pruebas

#### Función en módulo del proyecto

In [21]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [22]:
# Mido las distancias

#### Crear función para las distancias, apply lambda?

In [23]:
%%time
distance=[]
for index, row in cleaning_df_garden.iterrows():
    for index, j in df_bicimad.iterrows():
        distance.append(distance_meters(float(row['latitude']), float(row['longitude']), float(j['latitude']), float(j['longitude'])))

CPU times: total: 10min 24s
Wall time: 11min 43s


Función para agrupar en lista de listas

In [24]:
%%time
list_df_coordinates=[]
count=0
list_list = []
for i in distance:
    if count != 264:
        count = count + 1
        list_list.append(i)
    if count == 264: # = rows of stations --> df_bicimad(rows)
        count = 0
        list_df_coordinates.append(list_list)
        list_list = []

CPU times: total: 15.6 ms
Wall time: 18.4 ms


In [25]:
df_coordinates = pd.DataFrame(list_df_coordinates)
#df_coordinates

#### Extraer la función a módulos. Es para dejar solo números en data frame

In [32]:
%%time
def remove_non_numeric(s):
    return re.sub('[^0-9\.]+', '', s)

# convert all values in the dataframe to strings
df_coordinates = df_coordinates.astype(str)

# apply the function to all elements in the dataframe using a lambda function
df_coordinates = df_coordinates.applymap(lambda x: remove_non_numeric(x))

# convert all values in the dataframe back to float data type
df_coordinates = df_coordinates.astype(float)

CPU times: total: 1.59 s
Wall time: 18 s


In [ ]:
# mínimo de las filas
min_rows = df_coordinates.min(axis=1)

In [ ]:
dict_min_distances = min_rows.to_dict()

In [ ]:
dict_all_distances = df_coordinates.to_dict()

#### Extraer esta funcion a módulos, es para crear un diccionario con la estacion y la distancia, y asignar el indice de gardens como key

In [ ]:
dict_min_station = {}
for keys1, values1 in dict_all_distances.items():
    for key_a, value_a in values1.items():
        for key_min, value_min in dict_min_distances.items():
            if value_a == value_min:
                dict_min_station[key_min] = (value_min, keys1)
                
len(dict_min_station)

In [ ]:
dict_min_station = dict(sorted(dict_min_station.items())) # place: distance,station

In [ ]:
df_min_station=pd.DataFrame.from_dict(dict_min_station, orient='index')

In [ ]:
df_min_station

In [ ]:
df_min_station.rename(columns={1: "1a"}, inplace=True)

In [ ]:
df_bicimad['index'] = df_bicimad.index


In [ ]:
df_bicimad.rename(columns={"index": "1a"}, inplace=True)


#### Extraer esta función a módulos, es para asignar nombre de estacion a la fila

In [ ]:
list_min_station_names = {}
count = 0
for index, row1 in df_min_station.iterrows():
    for index, row2 in df_bicimad.iterrows():
        if row1['1a'] == row2['1a']:
            list_min_station_names[count]= (row2['name'], row2['address'])
            count = count +1


Tansformo la lista de estaciones con la mínima distancia

In [ ]:
df_min_station_names = pd.DataFrame(list_min_station_names)

Transpose para tener mismas filas que gardens

In [ ]:
df_min_station_names = df_min_station_names.transpose()

Renombro las columnas de las estaciones con la distancia mínima

In [ ]:
df_min_station_names.rename(columns={0: "name1", 1:'address1'}, inplace=True)

Split para separar numero de nombre

In [ ]:
df_min_station_names1 = df_min_station_names['name1'].apply(lambda x: pd.Series(str(x).split('-')))

Renombro las columnas

In [ ]:
df_min_station_names1.rename(columns={0: "numberof", 1:'name2', 2:'null'}, inplace=True)

Concateno los 3 data frames

In [ ]:
df_final = pd.concat([cleaning_df_garden, df_min_station_names, df_min_station_names1], axis=1)

Selecciono las columnas del df que concateno

In [ ]:
df_final = df_final[['title', '@type', 'street-address', 'name2', 'address1']]

Renombro las columnas para el resultado final que se pide

In [ ]:
df_final = df_final.rename(columns={'title': 'Place of interest', '@type': 'Type of place', 'street-address': 'Place address', 'name2': 'BiciMAD station', 'address1': 'Station location'})

In [ ]:
df_final.head()

Cambiar los null por Jardínes

In [ ]:
df_final["Type of place"].fillna("ParquesJardines", inplace = True)

In [ ]:
# Separar con espacios tipo de lugar

In [ ]:
'''It must create, at least, a .csv file including the requested table (i.e. Main Challenge). 
Alternatively, you may create an image, pdf, plot or any other output format that you may find convenient. 
You may also send your output by e-mail, upload it to a cloud repository, etc.'''

In [ ]:
def result(dff):
    dff.to_csv('./data/gardens.csv')  

In [ ]:
result(df_final)

Imports modules

In [ ]:
import argparse
from p_acquisition import m_acquisition as mac
from p_wrangling import m_wrangling as mwr
from p_analysis import m_analysis as man 
from p_reporting import m_reporting as mre


argument parser

In [ ]:
def argument_parser():
    parser = argparse.ArgumentParser(description = 'Place or type of places')
    parser.add_argument("-o", "--one", help="Select a place", action="store_true")
    parser.add_argument("-t", "--type", help="Select type of place", action="store_true")
    args = parser.parse_args()
    return args

In [ ]:
def argument_parser():
    parser = argparse.ArgumentParser(description = 'Set chart type')
    parser.add_argument("-b", "--bar", help="Produce a barplot", action="store_true")
    parser.add_argument("-l", "--line", help="Produce a lineplot", action="store_true")
    args = parser.parse_args()
    return args

main

In [ ]:
def main(some_args):
    data
    wrangling
    distances
    wrangling
    csv

In [ ]:
def main(some_args):
    data = mac.acquire()
    filtered = mwr.wrangle(data, year)
    results = man.analyze(filtered)
    fig = mre.plotting_function(results, title, arguments)
    mre.save_viz(fig, title)
    print('============= Pipeline is complete. You may find the results in the folder ./data/results ==============')

if __name__ == '__main__'

In [ ]:
if __name__ == '__main__':
    place = input('Enter the place: ')
    places = input('Enter the type of place: ')
    title = 'The station near of the place'
    arguments = argument_parser()
    main(arguments)

In [ ]:
if __name__ == '__main__':
    year = int(input('Enter the year: '))
    title = 'Top 10 Manufacturers by Fuel Efficiency ' + str(year)
    arguments = argument_parser()
    main(arguments)